In [1]:
# 山西网址： http://fgsjk.sxpc.gov.cn:8443/document-query
"""
列报数据爬取
"""
import math
import requests

import pandas as pd
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'http://fgsjk.sxpc.gov.cn:8443',
    'Pragma': 'no-cache',
    'Referer': 'http://fgsjk.sxpc.gov.cn:8443/document-query',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
}


def get_page_data(page_num: int = 1):
    json_data = {
        'pageNum': page_num,
        'pageSize': 40,
        'total': 4038,
        'searchField': 'title',
        'searchValue': '',
        'searchMode': 'like',
        'orderByColumn': '_score',
    }

    response = requests.post(
        'http://fgsjk.sxpc.gov.cn:8443/prod-api/extranet/regulation/search',
        headers=headers,
        json=json_data,
        verify=False,
    )
    resp_json = response.json()
    if resp_json.get('code') != 200:
        raise Exception('请求失败')
    current_page = resp_json['data']['pageNo']
    total_page = math.ceil(resp_json['data']['total']/resp_json['data']['pageSize'])
    return {
        'current_page': current_page,
        'total_page': total_page,
        'data': [{
            'id': _['id'],
            'title': _['title'],
            'file_type': _['filetypeVo']['name'],  # 文件类型
            'timeliness': _['timeliness'],  # 时效性
            'file_id': _['regulationFiles'][0]['attId'] if _['regulationFiles'] else None
        } for _ in resp_json['data']['rows']],
    }


if __name__ == '__main__':
    all_data = []
    res = get_page_data(1)
    all_data += [{**_, 'page': res['current_page']} for _ in res['data']]
    for p in range(2, res['total_page'] + 1):
        res = get_page_data(p)
        print(f'page {p}/{res["total_page"]}')
        print(res)
        all_data += [{**_, 'page': res['current_page']} for _ in res['data']]
    df = pd.DataFrame(all_data)
    df.to_csv('list_new.csv', index=False)

page 2/101
{'current_page': 2, 'total_page': 101, 'data': [{'id': '291cb9e95b80156cd1b1082b1898b9b2', 'title': '太原市小店区人民政府关于废止《关于印发太原市小店区创新提升营商环境加快推进市场主体倍增工程行动方案的通知》的通知', 'file_type': '县（市、区）政府及办公室规范性文件', 'timeliness': '现行有效', 'file_id': '992ff8bc50c2dd539cd6ee55eb59c976'}, {'id': '6f78591ac2183c5e2ad98c3bdc3e0c59', 'title': '关于加强经营主体集群登记管理工作的通知', 'file_type': '市政府部门行政规范性文件', 'timeliness': '现行有效', 'file_id': 'c82bb1e87f338b1e73711e45fdcb2d89'}, {'id': '405c363757286cc9c39574e90f4cfd4b', 'title': '乡宁县人民政府办公室 关于印发乡宁县企业资金链应急周转保障 资金管理办法(修订)的通知', 'file_type': '县（市、区）政府及办公室规范性文件', 'timeliness': '现行有效', 'file_id': 'be8db8a12ddfd671d90a1dc59121cb09'}, {'id': '0994bdfd578f2e3cce766cd84b28b5e9', 'title': '晋城市城区人民政府办公室关于印发晋城市城区加快特色餐饮（街区）建设的若干政策的通知', 'file_type': '县（市、区）政府及办公室规范性文件', 'timeliness': '现行有效', 'file_id': 'f5b977d03dc07e138ff453ab95f3ac03'}, {'id': 'a1a653744df9c2ac57dc538c02b6ad65', 'title': '太原市住房公积金管理中心 《关于优化加装电梯提取住房公积金使用政策的通知》', 'file_type': '市政府部门行政规范性文件', 'timeliness': '现行有效', 'fi